In [ ]:
# importing modules and functions to get data from the database
!pip install mysql.connector
import pandas as pd
pd.set_option('display.max_columns', None)
import mysql.connector
import numpy as np
pd.set_option('display.max_rows', 500)

class MysqlIO:
    """Connect to MySQL server with python and excecute SQL commands."""
    def __init__(self, database):
        try:
            connection = mysql.connector.connect(host='db.americor.consultancy.cmlinsight.com',
                                                 database=database,
                                                 user='admin',
                                                 password='admin@123',
                                                 use_pure=True
                                                 )
            if connection.is_connected():
                db_info = connection.get_server_info()
                print("Connected to MySQL Server version", db_info)
                print("Your're connected to database:", database)
                self.connection = connection
        except Exception as e:
            print("Error while connecting to MySQL", e)
            
    def execute(self, query, header=False):
        """Execute SQL commands and return retrieved queries."""
        cursor = self.connection.cursor(buffered=True)
        cursor.execute(query)
        try:
            record = cursor.fetchall()
            if header:
                header = [i[0] for i in cursor.description]
                return {'header': header, 'record': record}
            else:    
                return record
        except:
            pass
        
    def to_df(self, query):
        """Return the retrieved SQL queries into pandas dataframe."""
        res = self.execute(query, header=True)
        df = pd.DataFrame(res['record'])
        df.columns = res['header']
        return df

# Database Connection and merging necessary tables

In [ ]:
analytics_database = MysqlIO('americor_analytics')

In [ ]:
enroll_deposite_df  = analytics_database.to_df('SELECT * from mvd_fe_customer_profile_enroll_deposit order by customer_id ')

In [ ]:
enroll_deposite_df.info(verbose=True)

In [ ]:
Cancelled_df  = analytics_database.to_df('SELECT * from mvd_customer_lifecycle_full WHERE customer_type_value = "cancel";')

In [ ]:
import datetime
t_date = datetime.datetime(2018, 5, 1)
print(t_date)

In [ ]:
#Droping other coloumns from lifecycle table before merging
Cancelled_df = Cancelled_df[['customer_id','customer_type_event_ts','customer_type_value']]

In [ ]:
enroll_deposite_df['enroll_deposit_ts']=pd.to_datetime(enroll_deposite_df['enroll_deposit_ts'])

In [ ]:
#selecting entries which belongs to date After 2k18 May
Cancelled_df= Cancelled_df[Cancelled_df['customer_type_event_ts'] > t_date]
enroll_deposite_df= enroll_deposite_df[enroll_deposite_df['enroll_deposit_ts'] > t_date]

In [ ]:
merged_data=pd.merge(enroll_deposite_df, Cancelled_df, on=['customer_id'], how='left')

In [ ]:
merged_data.rename(columns = {'customer_type_value':'cancel'}, inplace = True)
merged_data.rename(columns = {'customer_type_event_ts':'cancel_ts'}, inplace = True)

In [ ]:
merged_data['cancel'] = merged_data['cancel'].fillna(0)
merged_data['cancel'] =  merged_data['cancel'].replace(['cancel'],1)

In [ ]:
merged_data.shape

# Creating nre features (Age)

In [ ]:
merged_data['dob']= pd.to_datetime(merged_data['dob'])

In [ ]:
merged_data['dob'].replace(r'^\s*$', np.nan, regex=True,inplace=True)

In [ ]:
merged_data['dob']=merged_data['dob'].fillna(merged_data['dob'].mode()[0])

In [ ]:
merged_data['Age'] = merged_data['enroll_deposit_ts'].dt.year-merged_data['dob'].dt.year

In [ ]:
merged_data.drop('dob',axis=1,inplace=True)

# data Inspection and columns droping

In [ ]:
data=merged_data.copy()

In [ ]:
data['cancel'].value_counts() 

In [ ]:
cancel_rate=32427/len(data)
cancel_rate

In [ ]:
# Remove completely null columns
def get_null_cols(df):
    null_cols = set()
    num_records = len(df)
    for col in df.columns:
        if df[col].isna().sum() == num_records:
            null_cols.add(col)
    return null_cols

null_cols = get_null_cols(data)
print(len(null_cols))
print(null_cols)

In [ ]:
# ids and timestamps are to be removed
manual_drop_cols = ['customer_id','lead_ts','primary_applicant_id','prev_event_ts','cancel_ts','enroll_ts','enroll_deposit_ts','email','edcp_credit_report_ts','edcp_prev_event_ts',
            #features with confirmed leakage
            'edex_extract_date', 'quality', 'detailed_hardship_reason','edex_ZIP_CD','budget_note','schedule_payments_type'
]  

In [ ]:
data.drop(null_cols,axis=1,inplace=True)
data.drop(manual_drop_cols, axis=1,inplace=True)

In [ ]:
#replacing null for empty strings
for i in data.columns:
    data[i].replace(r'^\s*$', np.nan, regex=True,inplace=True)

In [ ]:
# handle outliers
def handle_outliers(df, cols, thresholds):
    """
    For given set of columns, there are thresholds defined to identify outliers. The thresholds are passed
    as a parameter. If a column value of a particular row is larger than the threshold, the value is marked
    as None.
    :param cols: list of column names
    :param thresholds: list of threshold for each of column specified in the list of column names.
    """
    
    for i in range(len(cols)):
        for j in range(len(df)):
            if df[cols[i]].iloc[j] >= thresholds[i]:
                df[cols[i]].iloc[j] = None
               
                
    return df

cols_with_outliers = ['edex_P13_ALL7517', 'edex_P13_BCA7600', 'edex_P13_BCA8122', 'edex_P13_BCC3520', 'edex_P13_BCC5627',
                      'edex_P13_BCC7140', 'edex_P13_BCC7141', 'edex_P13_PIL5020', 'edex_P13_REH0437', 'edex_P13_REH5530',
                      'edex_UNSCD_RATIO_V1']
outlier_thresholds = [990, 990, 9900, 90, 999999990, 990, 990, 999999990, 999999990, 999999990, 100]
data = handle_outliers(data, cols_with_outliers, outlier_thresholds)

In [ ]:
n_samples = data.shape[0]
dropped_cols_90 = []
for col in data.columns:
    null_frac = data[col].isnull().sum() / n_samples
    if null_frac >= 0.9:
        dropped_cols_90.append(col)

In [ ]:
len(dropped_cols_90)

In [ ]:
dropped_cols_90

In [ ]:
data.drop(dropped_cols_90, axis=1, inplace=True) #droped columns with 90% Null value

In [ ]:
data['edex_EMAIL_IND'].value_counts()

In [ ]:
#Y and N coloumns are marked as null so convert as 0 and 1 to make use of this feature
data['edex_EMAIL_IND'] = data['edex_EMAIL_IND'].fillna(0)
data['edex_EMAIL_IND'] = data['edex_EMAIL_IND'].replace(['Y'],1)

In [ ]:
def drop_no_info_cols(df):
    """
    Drop columns that include only one value for all the rows
    :param df: dataframe
    :return df: processed df
    """
    unique_count = df.nunique()
    unique_check = unique_count[unique_count == 1]
    df_cols = df.columns
    cols_to_drop = df_cols[unique_count == 1]
    return cols_to_drop
  

In [ ]:
no_info_cols = drop_no_info_cols(data)

In [ ]:
data.drop(no_info_cols,axis=1,inplace=True) #removing coloumns that have only 1 valur throughout the coloumn

In [ ]:
data.info(verbose=True)

In [ ]:
def fill_all_nulls(df):
    data = df.copy()
    n_samples = data.shape[0]
    for col in data.columns:
        null_frac = data[col].isnull().sum() / n_samples
        if null_frac > 0:
          if data[col].dtypes == 'object':
            data[col] = data[col].fillna('Unknown')
          else:
            mean_val = np.mean(data[col])
            data[col] = data[col].fillna(mean_val)
    return data

data = fill_all_nulls(data)


In [ ]:
Numerical_cols=[]
obje_cols=[]
for col in data.columns:
  if data[col].dtypes == 'object':
    obje_cols.append(col)
  else:
    Numerical_cols.append(col)

In [ ]:
obje_cols

# MOM score value

In [ ]:
import datetime
import warnings
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', None)

In [ ]:
def columns_without_targets(data):
    colmns_without_targets = list(data.columns)
    #colmns_without_targets.remove('enroll_flag_timestamp')
    colmns_without_targets.remove('cancel')
    return colmns_without_targets

In [ ]:
def get_categorical_mom_score_with_na(mom_df, feature, outcome_variable):
    
    not_null_mom_df = mom_df[~mom_df[feature].isnull()]
    null_mom_df = mom_df[mom_df[feature].isnull()]

    not_null_1_df = not_null_mom_df[not_null_mom_df[outcome_variable] == 1].groupby(feature)[outcome_variable].count().to_frame()
    not_null_1_df = not_null_1_df.reset_index()
    not_null_1_df.columns = ['feature', '1_count']    
    
    null_1_count = null_mom_df[null_mom_df[outcome_variable] == 1].shape[0]
    
    not_null_0_df = not_null_mom_df[not_null_mom_df[outcome_variable] == 0].groupby(feature)[outcome_variable].count().to_frame()
    not_null_0_df = not_null_0_df.reset_index()
    not_null_0_df.columns = ['feature', '0_count']    
    
    null_0_count = null_mom_df[null_mom_df[outcome_variable] == 0].shape[0]
    
    count_df = not_null_1_df.merge(not_null_0_df, how='outer', on='feature')
    count_df['feature'] = count_df['feature'].apply(str)    
    
    null_count_dict = {'feature':'Null', '1_count': null_1_count,  '0_count': null_0_count}
    count_df = count_df.append(null_count_dict, ignore_index=True)   
    
    count_df = count_df.fillna(0)
    
    sum_1 = count_df['1_count'].sum()
    sum_0 = count_df['0_count'].sum()    

    count_df['1_frac'] = count_df['1_count']/sum_1
    count_df['0_frac'] = count_df['0_count']/sum_0  
    
    count_df['overlap'] = count_df.apply(lambda row: min(row['1_frac'], row['0_frac']), axis=1)
    
    mom_score_with_na = round(count_df['overlap'].sum(), 2)
    
    return mom_score_with_na

In [ ]:
def get_numerical_mom_score_with_na(fm,feature,target,Nbins):
    #--let's add NA
    warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
    persisted_id = fm[target] == 1
    not_persisted_id = fm[target] == 0
    Npersist = np.sum(persisted_id)
    Nnon_persist = np.sum(not_persisted_id)
    null_frac = fm[feature].isnull().sum()/len(fm[target])
    #print(feature, null_frac)
    #print(bin_min, bin_max)
    if fm[feature].dtypes == 'object':
        p00 = []
        p11 = []
        binn = []
        mom = []
    else:
        p1 = fm[feature][persisted_id]
        p0 = fm[feature][not_persisted_id]
        bin_max = np.max(fm[feature])
        bin_min = np.min(fm[feature])
        if null_frac > 0:
            delta_val = (bin_max - bin_min)/Nbins
            replace_val = bin_max + delta_val*5
            bin_max = replace_val
            tmp = fm[feature].fillna(replace_val)
            p1 = tmp[persisted_id]
            p0 = tmp[not_persisted_id]
        binn = np.linspace(bin_min,bin_max,Nbins)
        p00, bin0 = np.histogram(p0, binn)/(Nnon_persist*1.0)
        p11, bin1 = np.histogram(p1, binn)/(Npersist*1.0)
        mom = np.sum(np.minimum(p00,p11))
        mom = round(mom, 2)

    return mom

In [ ]:
mom_data= data.copy()

In [ ]:
# check mom score
drop_features_dict = {}
drop_features = []
feature_mom={}
for columnName in columns_without_targets(data):
    if data[columnName].dtypes == 'object':
        mom = get_categorical_mom_score_with_na(data, columnName, 'cancel')
   
    else:
        mom = get_numerical_mom_score_with_na(data, columnName,'cancel',30)
    
    if abs(mom)<=35e-2:
        drop_features.append(columnName)
        drop_features_dict[columnName]= mom 
    else:
        feature_mom[columnName]= mom 

In [ ]:
pd.DataFrame(drop_features_dict.items(), columns=['Dropped Feature', 'mom_score']).sort_values(by=['mom_score'])

In [ ]:
mom_score_df = pd.DataFrame(feature_mom.items(), columns=['Feature', 'mom_score']).sort_values(by=['mom_score'])

In [ ]:
mom_score_df

In [ ]:
#removing features baesd on the mom score. features having mom score less than 0.35 are removed
data.drop(drop_features,axis=1,inplace=True)

# Data preprocessing ctd..

In [ ]:
data.drop(['edex_CITY_NAME','zip'],axis=1,inplace=True) #cannot effectivly represent as numerical coloumns

In [ ]:
Numerical_cols=[]
obje_cols=[]
for col in data.columns:
  if data[col].dtypes == 'object':
    obje_cols.append(col)
  else:
    Numerical_cols.append(col)

In [ ]:
def encode_cateogrical_cols_v2(df):
    categorical_cols = df.select_dtypes(include='object')
    for col in categorical_cols:
        df[col], uniques = pd.factorize(df[col])

    return df

In [ ]:
data = encode_cateogrical_cols_v2(data)

# Model building

In [ ]:
df =data.copy()

In [ ]:
X = df.loc[:, df.columns != 'cancel']
y = df.loc[:, df.columns == 'cancel']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2350, stratify=y,shuffle=True)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
y_train['cancel'].value_counts()

In [ ]:
params = {
    'n_estimators': 100,
    'max_features': 0.33,
    'max_depth': 8,
    'random_state' : 0,
   }
rf_model = RandomForestClassifier(**params)
rf_model.fit(X_train, y_train)

In [ ]:
y_pred_prob = rf_model.predict_proba(X_test)

In [ ]:
y_pred = rf_model.predict(X_test)

In [ ]:
y_pred_prob

In [ ]:
y_pred_prob = y_pred_prob[:, 1]


In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test,  y_pred_prob)
auc = metrics.roc_auc_score(y_test, y_pred_prob)

In [ ]:
auc

In [ ]:
import seaborn as sns

auc_str = "{:0.3f}".format(auc)
sns.set(context="paper", font_scale=1.2)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, [0, 1], [0, 1])
plt.xlabel('False positives')
plt.ylabel('True positives')
plt.title('ROC curve with AUC = ' + auc_str)

In [ ]:
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc, precision_recall_curve, precision_score, \
    recall_score, confusion_matrix
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import OneHotEncoder
from sklearn.calibration import calibration_curve

In [ ]:
# Several key measurements are taken to measrure the model performance. Accuracy, precision, recall and f1 scores.
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Accuracy", accuracy)
print("Precision", precision)
print("Recall", recall)
print("F1", f1)

In [ ]:
## This is a helper function
def draw_plot(x_label, y_label, title):
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.show()

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_prob)
prc_auc = auc(recall, precision)
auc_str = "{:0.3f}".format(prc_auc)
baseline = y_test.sum() / len(y_test) * 1.0
plt.plot(recall, precision, [0, 1], [baseline, baseline])
draw_plot('Recall', 'Precision', 'PRC with AUC = ' + auc_str)

In [ ]:
rf_feature_importance = rf_model.feature_importances_

In [ ]:
rf_model.n_features_


In [ ]:
feature_names = list(df.columns)
feature_names.remove('cancel')

In [ ]:
feature_importance_df = pd.DataFrame.from_dict(dict(zip(feature_names, rf_feature_importance)), orient='index')

In [ ]:
feature_importance_df = feature_importance_df.reset_index()
feature_importance_df.columns = ['feature', 'importance']
feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)

In [ ]:
feature_importances = {'Feature':X_train.columns,'Feature importance':rf_model.feature_importances_}
df_feature_importances = pd.DataFrame(feature_importances)
df_feature_importances = df_feature_importances.sort_values(by=['Feature importance'], ascending=False)
df_feature_importances = df_feature_importances.merge(mom_score_df , on="Feature", how="left", indicator=False)
df_feature_importances

In [ ]:
def get_features_by_importance(classifier):
    """
    Get top 20 most of important features for the RF model to predict the label.
    """
    feature_importance_values = classifier.feature_importances_
    indices = np.argsort(feature_importance_values)[-20:]
    return feature_importance_values, indices

def draw_feature_importance(classifier, features):
    """
    Draw the level of importance of each feature for RF model to predict the label. Top 20 most importance
    features is considered

    :param features: list of feature names
    """
    feature_importance_values, indices = get_features_by_importance(classifier)
    plt.rcParams['font.size'] = '10'  # This does not work
    plt.title('Feature Importance')
    plt.barh(range(len(indices)), feature_importance_values[indices], color='b', align='center')
    plt.yticks(range(len(indices)), [features[i] for i in indices])
    plt.show()

In [ ]:
f=df.columns.drop('cancel')
draw_feature_importance(rf_model,f)

# Good-feature class-conditional distributions with MOM

In [ ]:
def class_cond_pdf2(fm,feature,target,Nbins):

    import warnings
    warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

    persisted_id = fm[target] == 1
    not_persisted_id = fm[target] == 0
    Npersist = np.sum(persisted_id)
    Nnon_persist = np.sum(not_persisted_id)
    if type(feature) == str:
        p1 = fm[feature][persisted_id] 
        p0 = fm[feature][not_persisted_id] 
        bin_max = np.max(fm[feature])
        bin_min = np.min(fm[feature])
    else:
        p1 = feature[persisted_id]
        p0 = feature[not_persisted_id]
        bin_max = np.max(feature)
        bin_min = np.min(feature)
    print(p0)
    print(Npersist)

    #print(bin_min, bin_max)
                 
    binn = np.linspace(bin_min,bin_max,Nbins)
    p00, bin0 = np.histogram(p0, binn)/(Nnon_persist*1.0)
    p11, bin1 = np.histogram(p1, binn)/(Npersist*1.0)
    mom = np.sum(np.minimum(p00,p11))
    plt.plot(binn[0:Nbins-1],p00, binn[0:Nbins-1],p11)
    momStr = "{:0.3f}".format(mom)
    if type(feature) == str:
        plt.title('PDFs for ' + feature + ' MOM = ' + momStr, fontsize = 16)
    else:
        plt.title('PDFs for prediction: ' + ' MOM = ' + momStr, fontsize = 16)
    plt.show()
    
    return p00, p11, binn[0:Nbins-1], mom

In [ ]:
top_features = list(df_feature_importances['Feature'].values)[:20]

In [ ]:
for feature in top_features[:10]:
    p00, binn, enroll_means, mom = class_cond_pdf2(data , feature ,"cancel", 30)